1. Read remapped bed file
2. Check that the alt base is actually the base in GRCh38
3. Switch name
4. Emit new bed file 

In [1]:
from pyfaidx import Fasta
import csv

In [12]:
bed_file="remap_data/remapped_GRCh37_GRCh38_flipped_sites.bed"
assm="/Users/deanna.church/Documents/Content/Assembly/GRCh38/GRCh38_ucsc.fna"#need to unzip file

In [21]:
h38=Fasta(assm, key_function = lambda x: x.replace('chr', ''))
h38['1'][835187]

>1:835188-835188
G

In [43]:
map_hash={
    'HSCHR11_2_CTG1' : '11_KI270902v1_alt',
    'HSCHR15_4_CTG8' : '15_KI270905v1_alt',
    'HSCHR17_2_CTG5' : '17_KI270908v1_alt',
    'HSCHR19_3_CTG2' : '19_GL383576v1_alt',
    'HSCHR1_2_CTG31' : '1_GL383519v1_alt',
    'HSCHR21_6_CTG1_1' : '21_KI270873v1_alt',
    'HSCHR4_RANDOM_CTG4' : '4_GL000008v2_random',
    'HSCHR6_MHC_SSTO_CTG1' : '6_GL000256v2_alt'
}

out=open('GRCh38_GRCh37_flipped_sites.bed', 'w')
out.write("track db=GRCh38\n")
with open(bed_file, 'r') as infile:
    data=csv.reader(infile, delimiter="\t")
    for line in data:
        if not line[0].startswith("#") and not line[0] == 'track':
            name=line[3]
            new_ref=line[3].split(':')[2].split('=')[1]
            new_alt=line[3].split(':')[1].split('=')[1]
            rsID=line[3].split(':')[0]
            if line[0] in map_hash.keys():
                try:
                    base=str(h38[map_hash[line[0]]][int(line[1])])
                except:
                    print "Map hash fail"
            else:
                try:
                    base=str(h38[line[0]][int(line[1])])
                except:
                    print "other fail"
                    
            if new_ref == base:
                out.write("%s\t%s\t%s\t%s:ref=%s:alt=%s\n" % (line[0], line[1], line[2], rsID, new_ref, new_alt))
            else:
                try:
                    print "RefBase mismatch: %s\t%s\t%s %s != %s" % (rsID, line[0], line[1], new_ref, base)
                except:
                    print "print fail"

out.close()

RefBase mismatch: rs3122962	22	11893547 T != A
RefBase mismatch: rs8186366	11	54821773 T != A
RefBase mismatch: rs6606806	15	22935422 A != T
RefBase mismatch: rs557965	HSCHR21_6_CTG1_1	142806 T != G
RefBase mismatch: rs6517816	HSCHR19_3_CTG2	73246 T != A
RefBase mismatch: rs2012713	21	7759127 G != T
RefBase mismatch: rs514626	21	6138244 C != G
RefBase mismatch: rs229354	21	6036218 C != G
RefBase mismatch: rs229358	21	5992604 A != T
RefBase mismatch: rs7277128	21	5161716 C != G
RefBase mismatch: rs9637178	21	5161709 G != C
RefBase mismatch: rs9982286	21	5112625 G != C
RefBase mismatch: rs412610	21	5090377 C != G
RefBase mismatch: rs413328	21	5088611 G != C
RefBase mismatch: rs178257	22	20968781 C != T
RefBase mismatch: rs603526	13	18421567 T != A


RefBase mismatch: rs3122962	22	11893547 T != A: this is secondary alignment
RefBase mismatch: rs8186366	11	54821773 T != A: sequence inverted in GRCh38, strand not in outputted bed file...
RefBase mismatch: rs6606806	15	22935422 A != T: sequence inverted in GRCh38, strand not in outputted bed file...
RefBase mismatch: rs557965	HSCHR21_6_CTG1_1	142806 T != G: not sure what is going on
RefBase mismatch: rs6517816	HSCHR19_3_CTG2	73246 T != A: sequence inverted in GRCh38, strand not in outputted bed file...
RefBase mismatch: rs2012713	21	7759127 G != T: not flipped

